# Share of organic area in total utilized agricultural area (UAA)

Reproducing data from Eurostat's article [Organic farming statistics](https://ec.europa.eu/eurostat/statistics-explained/index.php?title=Organic_farming_statistics)


In [1]:
import pandas as pd

In [2]:
# ORG_CROPAR 
# Organic crop area by agricultural production methods and crops (from 2012 onwards)
org_url = 'https://ec.europa.eu/eurostat/api/dissemination/sdmx/2.1/data/ORG_CROPAR$DEFAULTVIEW/A.HA.UAAXK0000.TOTAL.EU+EU27_2020+EU28+BE+BG+CZ+DK+DE+EE+IE+EL+ES+FR+HR+IT+CY+LV+LT+LU+HU+MT+NL+AT+PL+PT+RO+SI+SK+FI+SE+IS+NO+CH+UK+MK+RS+TR/?format=SDMX-CSV&startPeriod=2012&endPeriod=2020'
# TAG00025
# Utilised agricultural area by categories
uaa_url = 'https://ec.europa.eu/eurostat/api/dissemination/sdmx/2.1/data/TAG00025/A.UAA.MA.BE+BG+CZ+DK+DE+EE+IE+EL+ES+FR+HR+IT+CY+LV+LT+LU+HU+MT+NL+AT+PL+PT+RO+SI+SK+FI+SE+IS+LI+NO+CH+UK+ME+MK+AL+RS+TR+BA+XK/?format=SDMX-CSV&startPeriod=2009&endPeriod=2020'


In [3]:
df = pd.read_csv(org_url)

In [4]:
df.head()

,DATAFLOW,LAST UPDATE,freq,unit,crops,agprdmet,geo,TIME_PERIOD,OBS_VALUE,OBS_FLAG
0,ESTAT:ORG_CROPAR$DEFAULTVIEW(1.0),04/08/21 23:00:00,A,HA,UAAXK0000,TOTAL,AT,2012,533230,NaN
1,ESTAT:ORG_CROPAR$DEFAULTVIEW(1.0),04/08/21 23:00:00,A,HA,UAAXK0000,TOTAL,AT,2013,526689,NaN
2,ESTAT:ORG_CROPAR$DEFAULTVIEW(1.0),04/08/21 23:00:00,A,HA,UAAXK0000,TOTAL,AT,2014,525521,NaN
3,ESTAT:ORG_CROPAR$DEFAULTVIEW(1.0),04/08/21 23:00:00,A,HA,UAAXK0000,TOTAL,AT,2015,552141,NaN
4,ESTAT:ORG_CROPAR$DEFAULTVIEW(1.0),04/08/21 23:00:00,A,HA,UAAXK0000,TOTAL,AT,2016,571423,NaN


In [5]:
org_area = df.loc[df['TIME_PERIOD'] == 2019, ['geo', 'OBS_VALUE']].copy()

In [6]:
org_area.columns = ['geo', 'org_area']

In [7]:
org_area.set_index('geo', inplace=True)

In [8]:
org_area

,org_area
geo,
AT,671703
BE,93119
BG,117779
CH,169030
CY,6240
CZ,535185
DE,1290839
DK,291247
EE,220737
